In [187]:
from probabilityPlus import *
from prettytable import *
from agents import *
from ipythonblocks import *
from utils import *

In [238]:
T, F = True, False
burglary = BayesNet([
    ('Burglary', '', 0.001),
    ('Earthquake', '', 0.002),
    ('Alarm', 'Burglary Earthquake',
     {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ('JohnCalls', 'Alarm', {T: 0.90, F: 0.05}),
    ('MaryCalls', 'Alarm', {T: 0.70, F: 0.01})
    ])


In [239]:
E = ['Alarm', 'Earthquake', 'Burglary']

In [240]:
def desc_rec(front,exps,rede):
    if not front:
        return exps[1:]
    else:
        first=front[0]
        del front[0]
        filhos=rede.variable_node(first).children
        new = [f for f in filhos if f not in exps]
        front += new
        exps+=new
        return desc_rec(front,exps,rede)

def descendentes(var,rede):
    return desc_rec([var],[var],rede)

In [241]:
descendentes('Burglary',burglary)

['Alarm', 'JohnCalls', 'MaryCalls']

In [242]:
descendentes('Earthquake',burglary)


['Alarm', 'JohnCalls', 'MaryCalls']

In [243]:
descendentes('Alarm',burglary)


['JohnCalls', 'MaryCalls']

In [244]:
descendentes('JohnCalls',burglary)

[]

In [245]:
descendentes('MaryCalls',burglary)

[]

### Para X ser condicionalmente independente de Y:

- Y não pode ser descendente de X

- X não pode ser descendente de Y

- Nem X nem Y podem estar nem em E 

In [336]:
def cond_indep(X,Y,E,R):
    if Y in descendentes(X,R) or X in descendentes(Y,R) or Y in E or X in E:
        return False
    return True

def cond_indep_explica(X,Y,E,R):   
    #print("Para que X seja independente de Y dado E:")
    #print("• Y não pode ser descendente de X\n• X não pode ser descendente de Y\n• Nem X nem Y podem estar nem em E\n")
    print("E =",E)
    print("X =",X)
    print("Y =",Y)
    explicacao = "Por isso, X e Y não são condicionalmente independentes."
    if Y in descendentes(X,R):
        return (False,'Y é descendente de X. ' + explicacao)
    if X in descendentes(Y,R):
        return (False, 'X é descendente de Y. ' + explicacao)
    if Y in E:
        return (False, 'E contém Y. ' + explicacao)
    if X in E:
        return (False, 'E contém X. ' + explicacao)
    return (True, 'X e Y são condicionalmente independentes.')

def todos_cond_indeps(r):
    out=[]
    rVariables=set(r.variables)
    conjuntosE=itertools.combinations(iterable, r.variables)
    for x in rVariables:
        for y in rVariables:
            for e in conjuntosE:
                i=cond_indep({x},{y}, e,r)
                if i==True:
                    out.append({x},{y},e)
    return out

In [337]:
print(cond_indep_explica('Burglary', 'Alarm', 'Earthquake', burglary))

E = Earthquake
X = Burglary
Y = Alarm
(False, 'Y é descendente de X. Por isso, X e Y não são condicionalmente independentes.')


In [338]:
print(cond_indep_explica('JohnCalls', 'Alarm', 'Earthquake', burglary))

E = Earthquake
X = JohnCalls
Y = Alarm
(False, 'X é descendente de Y. Por isso, X e Y não são condicionalmente independentes.')


In [339]:
#testar os caso dos 2


In [340]:
print(cond_indep_explica('Burglary', 'Earthquake', E, burglary))

E = ['Alarm', 'Earthquake', 'Burglary']
X = Burglary
Y = Earthquake
(False, 'E contém Y. Por isso, X e Y não são condicionalmente independentes.')


In [341]:
print(cond_indep_explica('Burglary', 'Earthquake', 'Alarm', burglary))

E = Alarm
X = Burglary
Y = Earthquake
(True, 'X e Y são condicionalmente independentes.')


In [342]:
print("\nBurglary é independente de Alarm? dado pais(Burglary)?\n",cond_indep('Burglary', 'Alarm', 'Earthquake', burglary))


Burglary é independente de Alarm? dado pais(Burglary)?
 False


In [324]:
print("\nBurglary é independente de Earthquake? dado E?\n",cond_indep('Burglary', 'Earthquake', 'Alarm', burglary))



Burglary é independente de Earthquake? dado E?
 True


In [325]:
def display_cpt(node):
    header='P(' + node.variables[2] + '|' + ','.join(node.nodes[2].parents) + '):'
    #print(R)
    print(node.nodes[2], "\n")
    print(node.variables[2],"\n")
    print(node.nodes[2].parents,"\n")
    print(header,"\n")
    pretty=PrettyTable()
    aux = node.nodes[2].parents.copy()
    aux.append(node.variables[2])
    pretty.field_names = aux 
    #print(pretty.field_names)  # Caso Booleano #
    for i in list(node.nodes[2].cpt.keys()):
        #print(i)
        pretty.add_row(i+('True:'+str(node.nodes[2].cpt[i]),))
        pretty.add_row(i+('False:'+str(1.0-node.nodes[2].cpt[i]),))
    print(pretty)

In [68]:
display_cpt(burglary)

('Alarm', 'Burglary Earthquake') 

Alarm 

['Burglary', 'Earthquake'] 

P(Alarm|Burglary,Earthquake): 

+----------+------------+----------------------------+
| Burglary | Earthquake |           Alarm            |
+----------+------------+----------------------------+
|   True   |    True    |         True:0.95          |
|   True   |    True    | False:0.050000000000000044 |
|   True   |   False    |         True:0.94          |
|   True   |   False    | False:0.06000000000000005  |
|  False   |    True    |         True:0.29          |
|  False   |    True    |         False:0.71         |
|  False   |   False    |         True:0.001         |
|  False   |   False    |        False:0.999         |
+----------+------------+----------------------------+


In [44]:
i = 2
def linkedTo(n):
    pais = n.nodes[i].parents
    filhos = n.nodes[i].children
    display_cpt(n)
    print( pais , "--> ",n.variables[i] , "--> ", filhos)
    return ( pais, n.variables[i], filhos)
    # return (n.parents,n.variable,n.children)
linkedTo(burglary)

('Alarm', 'Burglary Earthquake') 

Alarm 

['Burglary', 'Earthquake'] 

P(Alarm|Burglary,Earthquake): 

+----------+------------+----------------------------+
| Burglary | Earthquake |           Alarm            |
+----------+------------+----------------------------+
|   True   |    True    |         True:0.95          |
|   True   |    True    | False:0.050000000000000044 |
|   True   |   False    |         True:0.94          |
|   True   |   False    | False:0.06000000000000005  |
|  False   |    True    |         True:0.29          |
|  False   |    True    |         False:0.71         |
|  False   |   False    |         True:0.001         |
|  False   |   False    |        False:0.999         |
+----------+------------+----------------------------+
['Burglary', 'Earthquake'] -->  Alarm -->  ['JohnCalls', 'MaryCalls']


(['Burglary', 'Earthquake'], 'Alarm', ['JohnCalls', 'MaryCalls'])